# Convert Norway dataset to the standard format of MBM

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import datetime as dt
import massbalancemachine as mbm

In [2]:
# Get filepath and filename of raw data
filepath = 'C:/Users/kasj/MassBalanceMachine/regions/Norway/data/'
filename = '2023-08-28_stake_mb_norway_cleaned_ids_latlon_wattributes_climate.csv'

# Read DataFrame
data = pd.read_csv(filepath + filename)

C:\Users\kasj\AppData\Local\Temp\ipykernel_24020\160331932.py:6: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filepath + filename)


In [3]:
# Drop columns not needed for MBM
drop_cols = ['GLIMSID','BREID', 'utm_zone', 'utm_east_approx', 'utm_north_approx', 'altitude_approx', 'location_description', 'location_id', 'stake_no',
             'utm_east', 'utm_north', 'stake_remark', 'flag_correction', 'approx_loc', 'approx_altitude', 'diff_north', 'diff_east', 'diff_altitude',
             'diff_netto', 'lat_approx', 'lon_approx', 'topo', 'slope_factor', 'dis_from_border']

data = data.drop(columns=drop_cols)

In [4]:
# Rename columns in WGMS format
data = data.rename(columns={'RGIID':'RGIId', 'altitude':'POINT_ELEVATION', 'lat':'POINT_LAT', 'lon':'POINT_LON', 'altitude_climate':'ALTITUDE_CLIMATE'})

In [5]:
# Get hydrological year from current year
data['YEAR']=pd.to_datetime(data['dt_curr_year_max_date'].astype('string'), format="%d.%m.%Y %H:%M")
data['YEAR'] = data.YEAR.dt.year.astype('Int64')

In [6]:
# Add measurement id to know which annual, summer, winter belong together
data['MEASUREMENT_ID'] = np.arange(0, len(data))

In [7]:
data["ELEVATION_DIFFERENCE"] = data["ALTITUDE_CLIMATE"] - data["POINT_ELEVATION"]
data

,RGIId,POINT_ELEVATION,balance_winter,balance_summer,balance_netto,dt_prev_year_min_date,dt_curr_year_max_date,dt_curr_year_min_date,POINT_LAT,POINT_LON,...,tp_apr,tp_may,tp_jun,tp_jul,tp_aug,tp_sep,ALTITUDE_CLIMATE,YEAR,MEASUREMENT_ID,ELEVATION_DIFFERENCE
0,RGI60-08.01258,471.0,1.32,-6.24,-4.92,25.09.2012 00:00,14.05.2013 00:00,07.11.2013 00:00,70.125889,21.777016,...,0.002334,0.001159,0.003054,0.003977,0.004691,0.001449,341.227919,2013,0,-129.772081
1,RGI60-08.01258,439.0,1.68,-4.35,-2.67,07.11.2013 00:00,09.05.2014 00:00,24.09.2014 00:00,70.125784,21.776300,...,0.005634,0.002244,0.003412,0.000891,0.002129,0.004860,341.227919,2014,1,-97.772081
2,RGI60-08.01258,444.0,0.76,-4.17,-3.41,24.09.2014 00:00,08.05.2015 00:00,23.09.2015 00:00,70.125732,21.775903,...,0.003739,0.004709,0.005111,0.001253,0.002506,0.003256,341.227919,2015,2,-102.772081
3,RGI60-08.01258,444.0,0.78,-4.37,-3.59,23.09.2015 00:00,23.05.2016 00:00,22.09.2016 00:00,70.125732,21.775903,...,0.001256,0.002129,0.003969,0.003559,0.002119,0.002851,341.227919,2016,3,-102.772081
4,RGI60-08.01258,468.0,1.59,-4.05,-2.46,22.09.2016 00:00,25.04.2017 00:00,29.09.2017 00:00,70.125536,21.773523,...,0.002704,0.003991,0.001780,0.004257,0.005193,0.000710,341.227919,2017,4,-126.772081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4166,RGI60-08.02969,1078.0,1.50,-3.57,-2.07,01.10.2012 00:00,03.05.2012 00:00,12.10.2012 00:00,59.938030,6.347331,...,0.003555,0.005019,0.004730,0.007226,0.005739,0.011397,917.419759,2012,4166,-160.580241
4167,RGI60-08.02969,1078.0,1.14,-4.29,-3.15,12.10.2012 00:00,29.05.2013 00:00,24.09.2013 00:00,59.937895,6.347288,...,0.009360,0.006135,0.006760,0.005158,0.008218,0.005461,917.419759,2013,4167,-160.580241
4168,RGI60-08.02969,1079.0,1.19,-3.80,-2.61,14.10.2014 00:00,22.04.2015 00:00,14.10.2015 00:00,59.938150,6.347017,...,0.006261,0.008175,0.005974,0.006563,0.008015,0.003896,917.419759,2015,4168,-161.580241
4169,RGI60-08.02969,1078.0,1.72,-4.38,-2.66,14.10.2015 00:00,28.04.2016 00:00,06.10.2016 00:00,59.938029,6.347241,...,0.004573,0.004449,0.005235,0.008926,0.011151,0.009556,917.419759,2016,4169,-160.580241


In [8]:
# Get separate dataframes for annual, winter and summer mass balance
data_annual = data[data['balance_netto'].notna()]
data_winter = data[data['balance_winter'].notna()]
data_summer = data[data['balance_summer'].notna()]

In [9]:
# Align dataset names
data_annual = data_annual.drop(columns=['balance_winter', 'balance_summer', 'dt_curr_year_max_date'])
data_annual = data_annual.rename(columns={'balance_netto':'POINT_BALANCE','dt_prev_year_min_date':'FROM_DATE','dt_curr_year_min_date':'TO_DATE'})

data_winter = data_winter.drop(columns=['balance_summer', 'balance_netto', 'dt_curr_year_min_date'])
data_winter = data_winter.rename(columns={'balance_winter':'POINT_BALANCE','dt_prev_year_min_date':'FROM_DATE','dt_curr_year_max_date':'TO_DATE'})

data_summer = data_summer.drop(columns=['balance_winter', 'balance_netto', 'dt_prev_year_min_date'])
data_summer = data_summer.rename(columns={'balance_summer':'POINT_BALANCE', 'dt_curr_year_max_date':'FROM_DATE', 'dt_curr_year_min_date':'TO_DATE'})

In [10]:
data_annual


,RGIId,POINT_ELEVATION,POINT_BALANCE,FROM_DATE,TO_DATE,POINT_LAT,POINT_LON,aspect,slope,u10_oct,...,tp_apr,tp_may,tp_jun,tp_jul,tp_aug,tp_sep,ALTITUDE_CLIMATE,YEAR,MEASUREMENT_ID,ELEVATION_DIFFERENCE
0,RGI60-08.01258,471.0,-4.92,25.09.2012 00:00,07.11.2013 00:00,70.125889,21.777016,0.629597,0.278773,-0.141265,...,0.002334,0.001159,0.003054,0.003977,0.004691,0.001449,341.227919,2013,0,-129.772081
1,RGI60-08.01258,439.0,-2.67,07.11.2013 00:00,24.09.2014 00:00,70.125784,21.776300,0.585924,0.296590,0.548836,...,0.005634,0.002244,0.003412,0.000891,0.002129,0.004860,341.227919,2014,1,-97.772081
2,RGI60-08.01258,444.0,-3.41,24.09.2014 00:00,23.09.2015 00:00,70.125732,21.775903,0.663261,0.302508,-0.709114,...,0.003739,0.004709,0.005111,0.001253,0.002506,0.003256,341.227919,2015,2,-102.772081
3,RGI60-08.01258,444.0,-3.59,23.09.2015 00:00,22.09.2016 00:00,70.125732,21.775903,0.663261,0.302508,0.841615,...,0.001256,0.002129,0.003969,0.003559,0.002119,0.002851,341.227919,2016,3,-102.772081
4,RGI60-08.01258,468.0,-2.46,22.09.2016 00:00,29.09.2017 00:00,70.125536,21.773523,0.836757,0.309196,0.955584,...,0.002704,0.003991,0.001780,0.004257,0.005193,0.000710,341.227919,2017,4,-126.772081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4166,RGI60-08.02969,1078.0,-2.07,01.10.2012 00:00,12.10.2012 00:00,59.938030,6.347331,2.906044,0.131717,-0.331500,...,0.003555,0.005019,0.004730,0.007226,0.005739,0.011397,917.419759,2012,4166,-160.580241
4167,RGI60-08.02969,1078.0,-3.15,12.10.2012 00:00,24.09.2013 00:00,59.937895,6.347288,2.906044,0.131717,-0.594287,...,0.009360,0.006135,0.006760,0.005158,0.008218,0.005461,917.419759,2013,4167,-160.580241
4168,RGI60-08.02969,1079.0,-2.61,14.10.2014 00:00,14.10.2015 00:00,59.938150,6.347017,2.906044,0.131717,-0.789949,...,0.006261,0.008175,0.005974,0.006563,0.008015,0.003896,917.419759,2015,4168,-161.580241
4169,RGI60-08.02969,1078.0,-2.66,14.10.2015 00:00,06.10.2016 00:00,59.938029,6.347241,2.906044,0.131717,-0.702830,...,0.004573,0.004449,0.005235,0.008926,0.011151,0.009556,917.419759,2016,4169,-160.580241


In [11]:
# Convert date format from DD.MM.YYYY HH:HH to 'YYYYMMDD'
data_annual['FROM_DATE']=pd.to_datetime(data_annual['FROM_DATE'], dayfirst=True)
data_annual['TO_DATE']=pd.to_datetime(data_annual['TO_DATE'], dayfirst=True)

data_annual['FROM_DATE'] = data_annual['FROM_DATE'].dt.strftime('%Y%m%d')
data_annual['TO_DATE'] = data_annual['TO_DATE'].dt.strftime('%Y%m%d')

data_summer['FROM_DATE']=pd.to_datetime(data_summer['FROM_DATE'], dayfirst=True)
data_summer['TO_DATE']=pd.to_datetime(data_summer['TO_DATE'], dayfirst=True)

data_summer['FROM_DATE'] = data_summer['FROM_DATE'].dt.strftime('%Y%m%d')
data_summer['TO_DATE'] = data_summer['TO_DATE'].dt.strftime('%Y%m%d')

data_winter['FROM_DATE']=pd.to_datetime(data_winter['FROM_DATE'], dayfirst=True)
data_winter['TO_DATE']=pd.to_datetime(data_winter['TO_DATE'], dayfirst=True)

data_winter['FROM_DATE'] = data_winter['FROM_DATE'].dt.strftime('%Y%m%d')
data_winter['TO_DATE'] = data_winter['TO_DATE'].dt.strftime('%Y%m%d')

In [12]:
#data_annual['POINT_ELEVATION'] = data_annual['POINT_ELEVATION'].astype(int)
#data_winter['POINT_ELEVATION'] = data_winter['POINT_ELEVATION'].astype(int)
#data_summer['POINT_ELEVATION'] = data_summer['POINT_ELEVATION'].astype(int)

In [13]:
data_winter['BALANCE_CODE']='BW'
data_summer['BALANCE_CODE']='BS'
data_annual['BALANCE_CODE']='BA'

In [14]:
data_cleaned = pd.concat([data_annual, data_winter, data_summer])

In [15]:
data_cleaned['POINT_ID'] = np.arange(0, len(data_cleaned))

In [16]:
data_cleaned.reset_index(inplace=True)
data_cleaned

,index,RGIId,POINT_ELEVATION,POINT_BALANCE,FROM_DATE,TO_DATE,POINT_LAT,POINT_LON,aspect,slope,...,tp_jun,tp_jul,tp_aug,tp_sep,ALTITUDE_CLIMATE,YEAR,MEASUREMENT_ID,ELEVATION_DIFFERENCE,BALANCE_CODE,POINT_ID
0,0,RGI60-08.01258,471.0,-4.92,20120925,20131107,70.125889,21.777016,0.629597,0.278773,...,0.003054,0.003977,0.004691,0.001449,341.227919,2013,0,-129.772081,BA,0
1,1,RGI60-08.01258,439.0,-2.67,20131107,20140924,70.125784,21.776300,0.585924,0.296590,...,0.003412,0.000891,0.002129,0.004860,341.227919,2014,1,-97.772081,BA,1
2,2,RGI60-08.01258,444.0,-3.41,20140924,20150923,70.125732,21.775903,0.663261,0.302508,...,0.005111,0.001253,0.002506,0.003256,341.227919,2015,2,-102.772081,BA,2
3,3,RGI60-08.01258,444.0,-3.59,20150923,20160922,70.125732,21.775903,0.663261,0.302508,...,0.003969,0.003559,0.002119,0.002851,341.227919,2016,3,-102.772081,BA,3
4,4,RGI60-08.01258,468.0,-2.46,20160922,20170929,70.125536,21.773523,0.836757,0.309196,...,0.001780,0.004257,0.005193,0.000710,341.227919,2017,4,-126.772081,BA,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11586,4166,RGI60-08.02969,1078.0,-3.57,20120503,20121012,59.938030,6.347331,2.906044,0.131717,...,0.004730,0.007226,0.005739,0.011397,917.419759,2012,4166,-160.580241,BS,11586
11587,4167,RGI60-08.02969,1078.0,-4.29,20130529,20130924,59.937895,6.347288,2.906044,0.131717,...,0.006760,0.005158,0.008218,0.005461,917.419759,2013,4167,-160.580241,BS,11587
11588,4168,RGI60-08.02969,1079.0,-3.80,20150422,20151014,59.938150,6.347017,2.906044,0.131717,...,0.005974,0.006563,0.008015,0.003896,917.419759,2015,4168,-161.580241,BS,11588
11589,4169,RGI60-08.02969,1078.0,-4.38,20160428,20161006,59.938029,6.347241,2.906044,0.131717,...,0.005235,0.008926,0.011151,0.009556,917.419759,2016,4169,-160.580241,BS,11589


In [17]:
# Get filepath and filename of raw data
filepath = 'C:/Users/kasj/MassBalanceMachine/regions/Norway/data/'
filename_save = '2024-09-11_stake_dataset_Norway.csv'

# Read DataFrame
data_cleaned.to_csv(filepath + filename_save)